In [3]:
import pandas as pd
import pdb
import numpy as np
import itertools
from utils import *
from sm_utils import *

from snakemake.io import expand
import yaml

In [4]:
config_file = 'config.yml'
with open(config_file) as f:
    config = yaml.safe_load(f)

## 240129

In [5]:
configfile: 'config.yml'
config_tsv = 'config.tsv'
p_meta_tsv = 'pseudochromosome_metadata.tsv'
meta_tsv = 'mouse_metadata.tsv'
auto_dedupe = True

df, p_df = parse_config_file(config_tsv,
                       meta_tsv,
                       p_meta_tsv,
                       auto_dedupe=auto_dedupe)
end_modes = ['tss', 'tes']
strands = ['fwd', 'rev']

In [7]:
expand(config['ref']['cerberus']['ends'],
               end_mode=end_modes)

['ref2/ca_tss.bed', 'ref2/ca_tes.bed']